In [2]:
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [16]:
from transformers import AutoTokenizer, pipeline, PretrainedConfig
from optimum.onnxruntime import ORTModelForSeq2SeqLM
import onnxruntime
from pathlib import Path

In [17]:
# Load encoder model
encoder_session = onnxruntime.InferenceSession('oyto_t5_small_onnx/encoder_model.onnx')

# Load decoder model
decoder_session = onnxruntime.InferenceSession('oyto_t5_small_onnx/decoder_model.onnx')

In [18]:
model_id = "oyto_t5_small_onnx/"
tokenizer = AutoTokenizer.from_pretrained(model_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
config = PretrainedConfig.from_json_file('oyto_t5_small_onnx/config.json')

In [20]:
model = ORTModelForSeq2SeqLM(
    config=config,
    onnx_paths=['oyto_t5_small_onnx/decoder_model.onnx','oyto_t5_small_onnx/encoder_model.onnx'],
    encoder_session=encoder_session, 
    decoder_session=decoder_session, 
    model_save_dir='oyto_t5_small_onnx',
    use_cache=False, 
)

In [21]:
onnx_translation = pipeline("translation_unyo_to_dcyo", model=model, tokenizer=tokenizer)

In [8]:
text_that_work_1 = "Bi a ba lo osuwon iseda, gbogbo gbolohun ede Yoruba ni a le pin si meji ni gbooro bayii"
text_that_work_2 = "Nje omo naa ni owo?"

In [22]:
def _extract_yoruba_sentences(file_path:str):
    """
    Extract only the Yoruba sentences from the jw300 text
    """
    yoruba_sentences = []

    with open(file_path, 'r', encoding='utf-8', newline='') as file:
        content = file.read()
        yoruba_sentences = content.strip().split('\n') # remove whitespaces, then break per line

    return yoruba_sentences

In [23]:
absolute_path = Path('/mnt/disk/makindele/data_prep_eng/data_prep_eng/output_data/test_with_no_diacritics.txt').resolve()
yoruba_sentences = _extract_yoruba_sentences(absolute_path)

In [24]:
yoruba_sentences

['Ijoba orileede Naijiria ni ijoba China lo n koju aarun korona kii se pe won koriira tabi saida si omo Naijiria tabi alawodudu to n gbe Guangzhou ni China.',
 'Leyin naa ni o koju si Labake pada...',
 'Awon alase ni baalu naa gbera lojo Eti laigba ase lati gbera kuro ni papako ofuurufu ni eyi t’o je ki ijoba ti papako Seattle-Tacoma pa.',
 'Oro to wa nile nisinsinyi yato',
 'Awon ni orile-ede meta ti, nigbeyin, won n gba awon atipo naa.',
 'Ka wi funni ka gbo; ka soro funni ka gba; a-wi-igbo, a-gbo-igba ni n fi igba adanu bu omi mu.',
 'Igbimo alase naa so pe ijoba apapo ti seleri lati fi egbe naa si ara awon ti yoo je anfaani bilionu marun un naa.',
 'Awon ibeere wonyi je atako-awon asikiri ati awon alatileyin omo ilu paraku, ti won sopo mo ipin onipele agbeka awon arale ati araata, awa ati awon, to je wi pe awa la se pataki julo, ti awon o si ri bee.',
 '"A ro gbogbo omo orile ede Naijiria lati duro sinu ile won, ki won si mojuto eto ilera won, nipa titele eto ilana ti ajo eto ilera

In [26]:
# olorun oba o, ese o
text = "Eleyii ni yoo je igba keta ti Omooba Wales naa yoo wa se ibewo ni orile-ede Naijiria, o koko wa si orile-ede Naijiria lodun 1990, 1999, ati odun 2006, sugbon igba akoko niyi fun Adari-obinrin ti ilu Cornwall lati wa si orile-ede Naijiria." # mo lo si ile
result = onnx_translation(text, max_length = 10000)
result[0]['translation_text']

'Èyí ni yóò jẹ́ ìgbà kẹta tí Ọmọọba Wales náà yóò wá ṣe ìbẹ̀wò ní orílẹ̀-èdè Nàíjíríà, ó kọ́kọ́ wá ṣe ìbẹ̀wò ní orílẹ̀-èdè Nàíjíríà, ó kọ́kọ́ wá sí orílẹ̀-èdè Nàíjíríà lọ́dún 1990, 1999, 1999, àti ọdún 2006, ṣùgbọ́n ìgbà àkọ́kọ́ nìyí fún Adari-obìnrin ti ìlú Cornwall láti wá sí orílẹ̀-èdè Nàìjíríà.'

In [33]:
for s in yoruba_sentences[:10]:
    result = onnx_translation(s, max_length = 10000)
    print(result[0]['translation_text'])

Ìjọba orílẹ̀-èdè Nàìjíríà ni ìjọba China ló ń kojú àrùn éèdì kì ṣe pé wọ́n kórìíra tàbí bẹnu àtẹ́ lu ọmọ Nàìjíríà tàbí aláwọ̀funfun tó ń gbé Guangzhou ní China.
Lẹ́yìn náà ni ó kọjú sí Labake padà...
alase ní baalu náà gbéra lọ́jọ́ Eti láìṣẹ̀ láti gbéra kúrò ní pápákọ̀ òfuurufú ní èyí tí wọ́n gbéra lọ́jọ́ Eti láìṣẹ̀ láti gbéra kúrò ní pápákọ̀ òfuurufú ní èyí tí ó jẹ́ kí ìjọba ti pápákọ̀ Seattle-Tacoma pa.
tó wà nílẹ̀ nísinsìnyí yàtọ̀ síra tó wà nílẹ̀ nísinsìnyí yàtọ̀
Àwọn ni orílẹ̀-èdè mẹ́ta ti, nígbẹ̀yìn, wọ́n ń gba àwọn àdàpọ̀ náà.
Ká wí fúnni ká gbọ́; ká sọ̀rọ̀ fúnni ká gbọ́; ká sọ̀rọ̀ fúnni ká gbọ́; ká sọ̀rọ̀ fúnni ká gbọ́; ká sọ̀rọ̀ fúnni ká gbọ́; ká sọ̀rọ̀ fúnni ká gbà; a-wí-ìgbó, a-gbó-gbà ni ń fi ìgbà ìpọ́njú bu omi mu.
ìgbìmọ̀ aláṣẹ náà sọ pé ìjọba àpapọ̀ ti ṣèlérí láti fi ẹgbẹ́ náà sí ara àwọn tí yóò jẹ́ àǹfààní bílíọ̀nù márùn ún náà.
ìyapa-àwọn apẹ̀yìndà àti àwọn alátìlẹ́yìn ọmọ ìlú paraku, tí wọ́n sopọ̀ mọ́ ìpín-àwọn apẹ̀yìndà àti àwọn alátìlẹ́yìn ọmọ ìlú paraku, tí wọ́n so